Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember,FullEnsemble

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Input parameters

In [ ]:
#year0 = 1851
year0 = 1951

option = ''
#option = '_fit'

Read data

In [ ]:
ad = AllData()
ad.year0 = year0
ad.option = option
ad.gather()

#Select subsection of models for rapid calculation
#ad.ds = ad.ds.isel(ism=slice(0,2),esm=slice(0,2))

fe = FullEnsemble(ad)
fe.gather(force_update=True)

try:
    fe = fe.drop_sel(ism='PISM_DMI')
except:
    pass

Figure settings

In [ ]:
plt.style.use('../src/style_paper')
mpl.rc('figure.subplot',left=.15)

bmps_long = ['linear basal melt','quadratic basal melt']
alpha = .3
perc = [17,83]

Make and save figure

In [ ]:
fig,ax = plt.subplots(2,3,sharex=True,sharey='row')

for b,bmp in enumerate(ad.bmps):
    for s,ss in enumerate(ad.ssp):
        dax = ax[b,s]
        dax.plot(fe.time,100*np.median(fe.slr_nf[b,s,:,:,:],axis=(0,1)),c='.5',lw=2,label='No feedback')
        dax.fill_between(fe.time,100*np.percentile(fe.slr_nf[b,s,:,:,:],perc[0],axis=(0,1)),100*np.percentile(fe.slr_nf[b,s,:,:,:],perc[1],axis=(0,1)),color='.5',alpha=alpha,label=perc)

        dax.plot(fe.time,100*np.median(fe.slr_wf[b,s,:,:,:],axis=(0,1)),c=ad.scol[ss],lw=2,label='With feedback')
        dax.fill_between(fe.time,100*np.percentile(fe.slr_wf[b,s,:,:,:],perc[0],axis=(0,1)),100*np.percentile(fe.slr_wf[b,s,:,:,:],perc[1],axis=(0,1)),color=ad.scol[ss],alpha=alpha)
        if b==0:
            dax.set_title(f'SSP-{ss}')
    ax[b,0].set_ylabel(bmps_long[b])
            
ax[0,0].legend(loc='upper left')
fig.supylabel('Sea level rise [cm]',fontsize=9)
ax[1,1].set_xlabel('Time [years]')

plt.savefig(f'../draftfigs/full_ensemble{ad.option}_{ad.year0}.png',dpi=450,facecolor='w',transparent=False)
plt.show()